### Proyecto Individual Henry
**5_Sistema_Reco**  
Sistema de Recomendación
**Autor: Bioing. Urteaga Facundo Nahuel**  

**Resumen:** Este script comprende las siguientes etapas:

1. **Carga de librerías**
2. **Primer entrenamiento del modelo (V1)**
3. **Segundo entrenamiento del modelo (V2)**
4. **Tercer entrenamiento del modelo (V3)**
5. **Cuarto entrenamiento del modelo (V4)**
6. **Exportar datos**

1. Carga de librerías

In [1]:
### 1. Carga de librerías

import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import warnings
warnings.filterwarnings('ignore')

2. Entrenamiento del primer modelo (V1)

In [2]:
### Modelo V1 ###
### Modelo inicial. El sistema de recomendación se elabora a partir de un algoritmo - K vecinos mas cercanos -
### Las variables de entrada son las etiquetas de cada juego en las categorías "genres", "specs" y "tags"

### 2.1 Carga de dataframes

df_games_tec = pd.read_parquet('df_games_tec.parquet')
df_games_genres = pd.read_parquet('df_games_genres.parquet')
df_games_specs = pd.read_parquet('df_games_specs.parquet')
df_games_tags = pd.read_parquet('df_games_tags.parquet')

### 2.2 Pre-procesamiento de dataframes para el posterior análisis (selección de variables y unión de tablas)

df_games_names = df_games_tec[['item_id', 'app_name']]
df_games_genres = df_games_genres.drop(columns=['genres'])
df_games_specs = df_games_specs.drop(columns=['specs'])
df_games_tags = df_games_tags.drop(columns=['tags'])

# Unión de los DF

merged_df_1 = pd.merge(df_games_names, df_games_genres, on='item_id', how='inner')
merged_df_2 = pd.merge(merged_df_1, df_games_specs, on='item_id', how='inner')
merged_df_final = pd.merge(merged_df_2, df_games_tags, on='item_id', how='inner')
#len(merged_df_final.columns)

### 2.3 Separo los nombres de los juegos de las variables de entrada del modelo

games_dummies = merged_df_final.drop(columns=['item_id', 'app_name'])
games_id_names = merged_df_final[['item_id', 'app_name']]

### 2.4 Entrenamiento del primer modelo (V1)

n_neighbors = 6
nneighbors = NearestNeighbors(n_neighbors = n_neighbors, metric = 'cosine').fit(games_dummies)

### Notas:
#           *El merge de los DF desordena las filas, lo que genera que un mismo juego tenga distintos índices por ejemplo entre df_games_tags y games_id_names.

### 2.5 TEST de modelo 

item_busqueda = 236390
index = games_dummies.index[games_id_names['item_id'] == item_busqueda][0]
#print(games_id_names["app_name"].iloc[index])

game_eval = np.array(games_dummies.iloc[index]).reshape(1,-1)
dif, ind = nneighbors.kneighbors(game_eval)

#print(df_games_names.loc[ind[0][0:], "app_name"].values)

print("Juego Seleccionado")
print("-"*80)
print(games_id_names["app_name"].iloc[index])
print("="*80)
print("Juegos Recomendados")
print("-"*80)
print(games_id_names.loc[ind[0][1:],  "app_name"])

Juego Seleccionado
--------------------------------------------------------------------------------
War Thunder
Juegos Recomendados
--------------------------------------------------------------------------------
1500                                          DCS World
28173              IL-2 Sturmovik: Battle of Stalingrad
6320     Combat Air Patrol 2: Military Flight Simulator
22756                                    Dogfight Elite
2356                                            Warface
Name: app_name, dtype: object


In [ ]:
### 2.6 Desempeño del modelo

# Indices de prueba. Juegos de diferentes características

#-------- JUEGO ---------|---id---|-Desempeño--------------
# -----------------------|--------|------------------------
# Counter Strike         |     10 | bien (Misma franquicia)
# PES 2018               | 592580 | regular                 
# AGE III                | 105450 | muy bien
# Simcity 4              |  24780 | muy bien
# Tennis Elbow 2013      | 346470 | regular
# Civilization IV        |  16810 | muy bien
# Darksiders             |  50620 | muy bien
# Fallout NV             |  22380 | bien (Misma franquicia)
# Dragon Age Origins     |  47810 | bien (Misma franquicia)
# Star Wars Jedi Knight  |   6020 | bien (Misma franquicia)
# NFS Shift              |  24870 | muy bien
# Final DOOM             |   2290 | bien (Misma franquicia)
# Earthworm Jim          | 901147 | muy bien

Entrenamiento del segundo modelo (V2)

In [3]:
### Modelo V2 ###
### Se detectan categorías en "specs", "genres" y "labels" que, a criterio del científico de datos, no aportan información relevante al algoritmo.
### Las columnas correspondientes a estas categorías se suprimen del DF
### Por otro lado, luego de estudiar la "objetividad" y la asignación de estas categorías a los juegos, se encuentra lo siguiente:
### "specs" es la categoría con las etiquetas mas fieles y concretas a cada juego
### "tags" es la categoría que aporta mayor ruido ya que las etiquetas son asignadas por los usuarios. Hay ejemplos que no corresponden a una correcta asignación,
### por ejemplo, un juego de fútbol (PES 2018) que tiene en tags la etiqueta "gore" o "heist"
### A partir de este análisis, se procede a ponderar penalizando o premiando estas categorías segun su objetividad, quedando genres*2 y tags*0.5

### 3.1 Carga de dataframes

df_games_tec = pd.read_parquet('df_games_tec.parquet')
df_games_genres = pd.read_parquet('df_games_genres.parquet')
df_games_specs = pd.read_parquet('df_games_specs.parquet')
df_games_tags = pd.read_parquet('df_games_tags.parquet')

### 3.2 Selección de categorías

df_games_names = df_games_tec[['item_id', 'app_name']]
df_games_genres = df_games_genres.drop(columns=['genres','Early Access'])
df_games_specs = df_games_specs[['item_id','Mods','Online Multi-Player','Standing','Local Multi-Player','Room-Scale',
    'Single-player', 'Windows Mixed Reality', 'Keyboard / Mouse','HTC Vive', 'Cross-Platform Multiplayer', 'Online Co-op', 'Seated',
    'MMO','Co-op', 'Gamepad', 'Downloadable Content','Local Co-op','Multi-player']]
df_games_tags = df_games_tags.drop(columns=['tags',"Early Access","Soundtrack"])

### 3.3 Pondero categorías

df_games_genres[df_games_genres == 1] = 2
df_games_specs[df_games_specs == 1] = 1
df_games_tags[df_games_tags == 1] = 0.5

### 3.4 Unión de tablas

merged_df_1 = pd.merge(df_games_names, df_games_genres, on='item_id', how='inner')
merged_df_2 = pd.merge(merged_df_1, df_games_specs, on='item_id', how='inner')
merged_df_final = pd.merge(merged_df_2, df_games_tags, on='item_id', how='inner')

### 3.5 Separo los nombres de los juegos de las variables de entrada del modelo

#games_dummies = merged_df_final.drop(columns=['item_id', 'app_name'])
games_dummies = merged_df_final.drop(columns=['item_id', 'app_name'])
games_id_names = merged_df_final[['item_id', 'app_name']]

### 3.6 Entrenamiento del segundo modelo (V2)

n_neighbors = 6
nneighbors = NearestNeighbors(n_neighbors = n_neighbors, metric = 'cosine').fit(games_dummies)

### 3.7 TEST de modelo 

item_busqueda = 236390
index = games_dummies.index[games_id_names['item_id'] == item_busqueda][0]
#print(games_id_names["app_name"].iloc[index])

game_eval = np.array(games_dummies.iloc[index]).reshape(1,-1)
dif, ind = nneighbors.kneighbors(game_eval)

#print(df_games_names.loc[ind[0][0:], "app_name"].values)

print("Juego Seleccionado")
print("-"*80)
print(games_id_names["app_name"].iloc[index])
print("="*80)
print("Juegos Recomendados")
print("-"*80)
print(games_id_names.loc[ind[0][1:],  "app_name"])


Juego Seleccionado
--------------------------------------------------------------------------------
War Thunder
Juegos Recomendados
--------------------------------------------------------------------------------
1500                                 DCS World
24243    Elite Dangerous: Horizons Season Pass
22756                           Dogfight Elite
27053                          Elite Dangerous
10043                         Toy Plane Heroes
Name: app_name, dtype: object


In [ ]:
### 3.8 Desempeño del modelo

# Indices de prueba. Juegos de diferentes características (Los simbolos ++ o -- indican un cambio positivo o negativo respecto al desempeño del modelo anterior)

#-------- JUEGO ---------|---id---|-Desempeño--------------
# -----------------------|--------|------------------------
# Counter Strike         |     10 | bien (Misma franquicia)
# PES 2018               | 592580 | regular
# AGE III                | 105450 | muy bien
# Simcity 4              |  24780 | muy bien
# Tennis Elbow 2013      | 346470 | regular
# Civilization IV        |  16810 | muy bien
# Darksiders             |  50620 | muy bien
# Fallout NV             |  22380 | muy bien ++
# Dragon Age Origins     |  47810 | muy bien ++
# Star Wars Jedi Knight  |   6020 | bien (Misma franquicia)
# NFS Shift              |  24870 | muy bien
# Final DOOM             |   2290 | muy bien ++
# Earthworm Jim          | 901147 | muy bien    

Entrenamiento del tercer modelo (V3)

In [4]:
### Modelo V3 ###
### Empíricamente se encuentra que al existir la etiqueta "Downloable Content" se recomiendan extensiones de juegos como DLCs, expansiones o contenido descargable 
### y no juegos completos lo que, a criterio del científico de datos, es indeseable por lo que se procede a borrar esta etiqueta.
### Además, se detecta la necesidad de relacionar el sistema de recomendación con el año de lanzamiento ya que, en general los usuarios tienden a consumir juegos de una misma época.
### Esto no es exclusivo ni determinante, pero puede ser una variable mas a tener en cuenta al momento del modelado del sistema para así obtener un mejor desempeño.
### Para este propósito, se categorizan en Lustros las fechas de lanzamiento y se ingresan al entrenamiento del modelo.

### 4.1 Carga de dataframes

df_games_tec = pd.read_parquet('df_games_tec.parquet')
df_games_genres = pd.read_parquet('df_games_genres.parquet')
df_games_specs = pd.read_parquet('df_games_specs.parquet')
df_games_tags = pd.read_parquet('df_games_tags.parquet')

### 4.2 Agrupo las fechas en décadas para que la influencia del año de cada juego sea mas flexible. Luego, genero variables dummies.

df_games_release_lustrum = df_games_tec[['item_id', 'release_year']].copy()
# Definir los límites de los lustros
bins = [0, 1999, 2005, 2010, 2015, 9999]
labels = ['before_2000', '2000_2005', '2005_2010', '2010_2015', 'after_2015']
# Dividir los años en lustros y crear variables dummies
df_games_release_lustrum['release_lustrum'] = pd.cut(df_games_release_lustrum['release_year'], bins=bins, labels=labels)
df_games_release_lustrum = pd.get_dummies(df_games_release_lustrum, columns=['release_lustrum'])
df_games_release_lustrum = df_games_release_lustrum.multiply(1)
# Eliminar la columna original de 'release_year'
df_games_release_lustrum.drop(columns=['release_year'], inplace=True)

### 4.3 Selecciono solo las columnas de interés

df_games_names = df_games_tec[['item_id', 'app_name']]
df_games_genres = df_games_genres.drop(columns=['genres','Early Access'])
df_games_specs = df_games_specs[['item_id','Online Multi-Player','Local Multi-Player','Room-Scale',
    'Single-player', 'Keyboard / Mouse', 'Cross-Platform Multiplayer', 'Online Co-op', 'Seated',
    'MMO','Co-op', 'Gamepad','Local Co-op','Multi-player']]
df_games_tags = df_games_tags.drop(columns=['tags',"Early Access","Soundtrack"])

### 4.4 Pondero categorías

df_games_release_lustrum[df_games_release_lustrum == 1] = 2 # Ponderación fuerte ya que es una sola columna con 1
df_games_genres[df_games_genres == 1] = 2
df_games_specs[df_games_specs == 1] = 1
df_games_tags[df_games_tags == 1] = 0.5

### 4.5 Unión de los DFs

merged_df_1 = pd.merge(df_games_names, df_games_genres, on='item_id', how='inner')
merged_df_2 = pd.merge(merged_df_1, df_games_specs, on='item_id', how='inner')
merged_df_3 = pd.merge(merged_df_2, df_games_release_lustrum, on='item_id', how='inner')
merged_df_final = pd.merge(merged_df_3, df_games_tags, on='item_id', how='inner')

### 4.6 Separo los nombres de los juegos de las variables de entrada del modelo

#games_dummies = merged_df_final.drop(columns=['item_id', 'app_name'])
games_dummies = merged_df_final.drop(columns=['item_id', 'app_name'])
games_id_names = merged_df_final[['item_id', 'app_name']]

### 4.7 Entrenamiento del tercer modelo (V3)

n_neighbors = 6
nneighbors = NearestNeighbors(n_neighbors = n_neighbors, metric = 'cosine').fit(games_dummies)

### 4.8 TEST de modelo 

item_busqueda = 236390
index = games_dummies.index[games_id_names['item_id'] == item_busqueda][0]
#print(games_id_names["app_name"].iloc[index])

game_eval = np.array(games_dummies.iloc[index]).reshape(1,-1)
dif, ind = nneighbors.kneighbors(game_eval)

#print(df_games_names.loc[ind[0][0:], "app_name"].values)

print("Juego Seleccionado")
print("-"*80)
print(games_id_names["app_name"].iloc[index])
print("="*80)
print("Juegos Recomendados")
print("-"*80)
print(games_id_names.loc[ind[0][1:],  "app_name"])

Juego Seleccionado
--------------------------------------------------------------------------------
War Thunder
Juegos Recomendados
--------------------------------------------------------------------------------
24259    Elite Dangerous: Horizons Season Pass
1500                                 DCS World
21277                                  Spectre
27069                          Elite Dangerous
22772                           Dogfight Elite
Name: app_name, dtype: object


In [ ]:
### 4.9 Desempeño del modelo

# Indices de prueba. Juegos de diferentes características (Los simbolos ++ o -- indican un cambio positivo o negativo respecto al desempeño del modelo anterior)

#-------- JUEGO ---------|---id---|-Desempeño--------------
# -----------------------|--------|------------------------
# Counter Strike         |     10 | muy bien ++
# PES 2018               | 592580 | bien ++
# AGE III                | 105450 | muy bien
# Simcity 4              |  24780 | muy bien
# Tennis Elbow 2013      | 346470 | regular (falta que recomiende otros juegos de tennis)
# Civilization IV        |  16810 | excelente ++
# Darksiders             |  50620 | excelente ++
# Fallout NV             |  22380 | bien -- (Agrego muchos de otra franquicia)
# Dragon Age Origins     |  47810 | excelente ++
# Star Wars Jedi Knight  |   6020 | muy bien ++
# NFS Shift              |  24870 | bien -- (OJO CON LOS EXPANSION PACK)
# Final DOOM             |   2290 | excelente ++
# Earthworm Jim          | 901147 | muy bien    

Entrenamiento del cuarto modelo (V4)

In [5]:
### Modelo V4 ###
### Se encuentran muchos juegos que en algunas categorías (Principalmente specs y tags) disponen de una gran cantidad de etiquetas.
### Esto puede "diluir" la influencia de las etiquetas de, por ejemplo, la categoría genre. Con el propósito de intentar mejorar el algoritmo en este sentido
### se propone en el modelo V4 dividir en cada categoría las etiquetas existentes (iguales a 1) por el total de etiquetas en cada categoría.
### Entonces, por ejemplo, si la categoría tags tiene 10 etiquetas, cada etiqueta pasaría de valer 1 a valer 0.1, penalizando así los juegos con una cantidad
### "exagerada" de etiquetas.

### 5.1 Carga de dataframes

df_games_tec = pd.read_parquet('df_games_tec.parquet')
df_games_genres = pd.read_parquet('df_games_genres.parquet')
df_games_specs = pd.read_parquet('df_games_specs.parquet')
df_games_tags = pd.read_parquet('df_games_tags.parquet')

### 5.2 Selecciono solo las columnas de interés

df_games_names = df_games_tec[['item_id', 'app_name']]
df_games_genres = df_games_genres.drop(columns=['genres','Early Access'])
df_games_specs = df_games_specs[['item_id','Online Multi-Player','Local Multi-Player','Room-Scale',
    'Single-player', 'Keyboard / Mouse', 'Cross-Platform Multiplayer', 'Online Co-op', 'Seated',
    'MMO','Co-op', 'Gamepad','Local Co-op','Multi-player']]
df_games_tags = df_games_tags.drop(columns=['tags',"Early Access","Soundtrack"])

### 5.3 Tratamiento de normalización de variables dummies

# Seleccionar solo las columnas de variables dummies
df_games_genres_dummies = df_games_genres.drop(columns=['item_id'])
df_games_specs_dummies = df_games_specs.drop(columns=['item_id'])
df_games_tags_dummies = df_games_tags.drop(columns=['item_id'])
# Sumar por fila la cantidad de variables dummies que son 1
suma_por_fila1 = df_games_genres_dummies.sum(axis=1)
suma_por_fila2 = df_games_specs_dummies.sum(axis=1)
suma_por_fila3 = df_games_tags_dummies.sum(axis=1)
# Dividir cada valor en la fila por la suma total (evitando la división por cero)
df_games_genres_dummies_dividido = df_games_genres_dummies.div(suma_por_fila1, axis=0)
df_games_specs_dummies_dividido = df_games_specs_dummies.div(suma_por_fila2, axis=0)
df_games_tags_dummies_dividido = df_games_tags_dummies.div(suma_por_fila3, axis=0)
# Reemplazar NaN con 0 si la suma por fila es 0
df_games_genres_dummies_dividido.fillna(0, inplace=True)
df_games_specs_dummies_dividido.fillna(0, inplace=True)
df_games_tags_dummies_dividido.fillna(0, inplace=True)
# Unir el DataFrame resultante con las columnas 'item_id' y 'genres'
df_games_genres_v4 = pd.concat([df_games_genres[['item_id']], df_games_genres_dummies_dividido], axis=1)
df_games_specs_v4 = pd.concat([df_games_specs[['item_id']], df_games_specs_dummies_dividido], axis=1)
df_games_tags_v4 = pd.concat([df_games_tags[['item_id']], df_games_tags_dummies_dividido], axis=1)

### 5.4 Agrupo las fechas en décadas para que la influencia del año de cada juego sea mas flexible. Luego, genero variables dummies.

df_games_release_lustrum = df_games_tec[['item_id', 'release_year']].copy()
# Definir los límites de los lustros
bins = [0, 1999, 2005, 2010, 2015, 9999]
labels = ['before_2000', '2000_2005', '2005_2010', '2010_2015', 'after_2015']
# Dividir los años en lustros y crear variables dummies
df_games_release_lustrum['release_lustrum'] = pd.cut(df_games_release_lustrum['release_year'], bins=bins, labels=labels)
df_games_release_lustrum = pd.get_dummies(df_games_release_lustrum, columns=['release_lustrum'])
df_games_release_lustrum = df_games_release_lustrum.multiply(1)
# Eliminar la columna original de 'release_year'
df_games_release_lustrum.drop(columns=['release_year'], inplace=True)

### 5.5 Ponderar categorías (Dejo de ponderar las demás ya que el proceso de penalizar la cantidad genera un efecto similar)

df_games_release_lustrum[df_games_release_lustrum == 1] = 2 # Ponderación fuerte ya que es una sola columna con 1
#df_games_specs_v4[df_games_specs_v4 == 1] = 0.25
#df_games_tags_v4[df_games_tags == 1] = 1
#df_games_genres_v4[df_games_genres == 1] = 0.125

### 5.6 Unión de los DFs

merged_df_1 = pd.merge(df_games_names, df_games_genres_v4, on='item_id', how='inner')
merged_df_2 = pd.merge(merged_df_1, df_games_specs_v4, on='item_id', how='inner')
merged_df_3 = pd.merge(merged_df_2, df_games_release_lustrum, on='item_id', how='inner')
merged_df_final = pd.merge(merged_df_3, df_games_tags_v4, on='item_id', how='inner')

### 5.7 Separo los nombres de los juegos de las variables de entrada del modelo

#games_dummies = merged_df_final.drop(columns=['item_id', 'app_name'])
games_dummies = merged_df_final.drop(columns=['item_id', 'app_name'])
games_id_names = merged_df_final[['item_id', 'app_name']]

### 5.8 Entrenamiento del cuarto modelo (V4)

n_neighbors = 6
nneighbors = NearestNeighbors(n_neighbors = n_neighbors, metric = 'cosine').fit(games_dummies)

### 5.9 TEST de modelo 

item_busqueda = 467620
index = games_dummies.index[games_id_names['item_id'] == item_busqueda][0]
#print(games_id_names["app_name"].iloc[index])

game_eval = np.array(games_dummies.iloc[index]).reshape(1,-1)
dif, ind = nneighbors.kneighbors(game_eval)

#print(df_games_names.loc[ind[0][0:], "app_name"].values)

print("Juego Seleccionado")
print("-"*80)
print(games_id_names["app_name"].iloc[index])
print("="*80)
print("Juegos Recomendados")
print("-"*80)
print(games_id_names.loc[ind[0][1:],  "app_name"])

Juego Seleccionado
--------------------------------------------------------------------------------
Baldur's Gate II: Enhanced Edition Official Soundtrack
Juegos Recomendados
--------------------------------------------------------------------------------
23640    Baldur's Gate: Siege of Dragonspear Official S...
6479     Baldur's Gate II: Enhanced Edition Official So...
10978                Baldur's Gate: Faces of Good and Evil
12676                          SpellForce 3 Digital Extras
15620                    Endless Legend™ - Digital Artbook
Name: app_name, dtype: object


In [12]:
games_id_names.at[ind[0][1], "app_name"]

Baldur's Gate: Siege of Dragonspear Official Soundtrack


AttributeError: 'str' object has no attribute 'value'

In [ ]:
### 5.10 Desempeño del modelo

# Indices de prueba. Juegos de diferentes características (Los simbolos ++ o -- indican un cambio positivo o negativo respecto al desempeño del modelo anterior)

#-------- JUEGO ---------|---id---|-Desempeño--------------
# -----------------------|--------|------------------------
# Counter Strike         |     10 | muy bien
# PES 2018               | 592580 | muy bien ++
# AGE III                | 105450 | muy bien
# Simcity 4              |  24780 | muy bien
# Tennis Elbow 2013      | 346470 | regular (falta que recomiende otros juegos de tennis)
# Civilization IV        |  16810 | excelente
# Darksiders             |  50620 | excelente
# Fallout NV             |  22380 | muy bien ++ (pueden faltar juegos de la franquicia)
# Dragon Age Origins     |  47810 | excelente
# Star Wars Jedi Knight  |   6020 | excelente ++
# NFS Shift              |  24870 | excelente ++
# Final DOOM             |   2290 | excelente
# Earthworm Jim          | 901147 | muy bien    

Exportar datos

In [6]:
### 6. Exporto en formato parquet el df final para implementar el Sistema de Recomendación V4

merged_df_final.to_parquet('df_sist_reco_v4.parquet')
